# Riddler: Fibonacci at the Post Office
https://fivethirtyeight.com/features/is-it-anyones-birthday/

_A postal worker and his customer joke about the various ways the customer could mathematically encode her post office box number. The customer realizes that every integer greater than 1 can be encoded via at least one Fibonacci-like sequence using an ordered triple (m, n, q). The encoded number is the qth member of the sequence after the first two positive integers m and n, where each term is the sum of the previous two terms. For example, 7 has the encodings (3, 4, 1) and (1, 3, 2). In an attempt to stump the postal worker, the customer prefers encodings with a maximal value of q. What encoding should she use for the number 81? **Extra credit:** What encoding should she use for the number 179?_

In [1]:
from itertools import product
from pprint import pprint

## Initial thoughts

The hard part of this problem will be determining how we know that we have the maximal value of q. It's clear that the minimal solution for a given number N is always 1. That is, the tuple (1, N-1, 1), i.e. the sequence 1, N-1, N, ..., always works. But given a solution, how do we know it is maximal? That is not immediately obvious to me, besides that n and m should be as small as possible.

My first thought is to go with a brute force solution. That is, for a number N (e.g. 81) generate all possible sequences with m,n < N, find all sequences that contain the number N and find the sequence with maximal q. This honestly seems a bit boring. Surely there is a slicker solution. I spent some time working on this problem on paper, trying to find something, but I didn't figure anything out 😢.

### But wait!
Ok, I did figure one thing out on paper that may make the brute force solution more efficient. After writing down some sequences, I noticed that a sequence defined by (m, n) can be written as the linear combination of two Fibonacci sequences.

**Lemma**: Let $m, n$ be positive integers. Then each Fibonacci-like sequence $G_q$ defined recursively as $G_1=m, G_2=n$ and $G_q=G_q+G_{q+1}$ can be written as a linear combination the Fibonacci sequence $F_q$ as $G_q = mF_{q-3}+nF_{q-2}$.

**Proof**: We can prove this inductively.

_Base Case_: We have previously defined $G_3=m+n,F_1=1$ and $F_2=1$. Therefore, $mF_1+nF_2=m+n=G_3$.

_Induction_: Suppose that $G_q = mF_{q-3}+nF_{q-2}$ and $G_{q-1}=mF_{q-4}+nF_{q-3}$. Then,

\begin{equation} \label{eq1}
\begin{split}
G_{q+1} & = G_q+G_{q-1} \\
& = mF_{q-3}+nF_{q-2} + mF_{q-4}+nF_{q-3} \\
& = m(F_{q-3}+F_{q-4}) + n(F_{q-2}+F_{q-3}) \\
& = mF_{q-2}+nF_{q-1}
\end{split}
\end{equation}

which completes the proof. $\blacksquare$

So what does that get us? This means that for a given number N, we only need to generate the Fibonacci sequence, and then we can generate all possible Fibonacci-like sequences by simple multiplication. Let's try it!

## Working on a solution 

In [2]:
# First, the fibonacci class
class Fibonacci:
    def __init__(self):
        self.cache = [0, 1]
        
    def __call__(self, n):
        if n < len(self.cache):
            return self.cache[n]
        else:
            fib_number = self(n - 1) + self(n - 2)
            self.cache.append(fib_number)
            
        return self.cache[n]


def post_office_sequence(m,n,length):
    """Returns a list of digits of the post office sequence, of the provided length"""
    fibonacci_number = Fibonacci()
    # Use max(m,n) as a reasonable max required length
    fibs = [m*fibonacci_number(i)+n*fibonacci_number(i+1) for i in range(1,length-2)]
    #initialize with the first three elements
    like_fibs=[0,m,n]
    like_fibs.extend(fibs)
    return like_fibs


def all_possible_inputs(max_coeff):
    """Returns a list of all possible (m,n) for m,n=1 through max(m,n)"""
    return list(product(range(1,max_coeff+1),repeat=2))


def all_sequences(max_coeff, length):
    """Returns a dictionary with input tuples as keys and post office sequences generated using those keys as values"""
    return {i:post_office_sequence(i[0],i[1],length) for i in all_possible_inputs(max_coeff)}


def maximal_q_encoding(q, length):
    """Find the encoding and sequence produced by the encoding for the sequence with maximal q"""
    sequence_data = [(coeff, sequence, q in sequence, sequence.index(q) if q in sequence else None) for coeff, sequence in all_sequences(q,length).items()]

    # Filter for sequences containing q and sort by index 
    filtered_sequence_data = list(filter(lambda s: s[2] == True, sequence_data))
    sorted_sequence_data = sorted(filtered_sequence_data, key=lambda sequence: sequence[3], reverse=True)
    return sorted_sequence_data[0]

### The Answer!

In [3]:
pprint(maximal_q_encoding(81,15))
# For q = 81, the maximal sequence is m = 3, n = 2 and 81 appears as the 9th digit of the sequence, or 6th digit if you are excluding 0, m and n as in the puzzle syntax.

pprint(maximal_q_encoding(179,15))
# For q = 179, the maximal sequence is m = 11, n = 7 and 81 appears as the 8th digit of the sequence, or 5th digit if you are excluding 0, m and n as in the puzzle syntax.

((3, 2), [0, 3, 2, 5, 7, 12, 19, 31, 50, 81, 131, 212, 343, 555, 898], True, 9)
((11, 7),
 [0, 11, 7, 18, 25, 43, 68, 111, 179, 290, 469, 759, 1228, 1987, 3215],
 True,
 8)
